In [ ]:
from ml_extra.decorators.mlflow.exp_tracking import mlflow_experiment 
from ml_extra.decorators.mlflow.exp_tracking import mlflow_client
from ml_extra.decorators.mlflow.exp_tracking import mlflow_tracking_uri

from prompteng.prompts import templates
from prompteng.utils.filesystem import get_root_path
import os 
import mlflow

In [ ]:
root_path = get_root_path()
os.environ["MLFLOW_TRACKING_URI"] = (root_path / "mlruns").as_uri()

In [ ]:
@mlflow_tracking_uri
@mlflow_experiment(name="simple_prompt_experiment")
def log_simple_prompt(**kwargs):
    """
    A simple function to demonstrate logging a prompt in an MLflow experiment.
    """

    template = kwargs.get("template")
    name = kwargs.get("name", "simple_prompt")
    with mlflow.start_run(run_name="simple_prompt_run"):
        mlflow.genai.register_prompt(
            name=name,
            template=template,
        )

In [ ]:
log_simple_prompt(template=templates.TOPIC_FINDER, name="topic_finder_prompt")

In [ ]:
loaded_template = mlflow.genai.load_prompt(name_or_uri ="topic_finder_prompt", version="4")

In [ ]:
print(loaded_template.format(
    text="Artificial Intelligence"
))

In [ ]:
from prompteng.prompts.openai_client import OpenAIClient
from dotenv import load_dotenv

In [ ]:
load_dotenv(dotenv_path = root_path / ".env")

In [ ]:
client = OpenAIClient()

In [ ]:
import json

In [ ]:
response = client.request(
    method="POST",
    endpoint="responses",
    data = json.dumps({"model":"gpt-4","input":"Tell me something about Artificial Intelligence"}))

In [ ]:
response.json()